In [1]:
import numpy as np

In [41]:
# and
w = np.array([0.5, 0.5])
b = np.array([0.6])
w = w.reshape(2, 1)

np.save('w.npy', w)
np.save('b.npy', b)

w1 = np.load('w.npy')
b1 = np.load('b.npy')

In [40]:
# nand
w2 = np.array([[-0.5], [-0.5]])
# b2 = np.array([-0.6])
b2 = np.array([0.6])

In [39]:
# or
w3 = np.array([[0.5], [0.5]])
b3 = np.array([0.4])

In [22]:
x1 = np.array([True, True, False, False])
x2 = np.array([True, False, True, False])
X = np.c_[x1, x2]
X = X.astype(int)

y_and = np.array(x1 & x2).reshape(-1, 1)
y_and = y_and.astype(int)

y_or = np.array(x1 | x2).reshape(-1, 1)
y_or = y_or.astype(int)

y_nand = np.array(~x1 | ~x2).reshape(-1, 1)
y_nand = y_nand.astype(int)

# x1 = 0
# x2 = 0
# X = np.c_[x1, x2]

# def and_predict(x1, x2, w1, w2, b):
def and_predict(x1, x2):
    X = np.c_[x1, x2]
    result = np.dot(X, w1) + b1
    if result >= 0:
        return 1
    else:
        return 0

def nand_predict(x1, x2):
    X = np.c_[x1, x2]
    # result = np.dot(X, w2) - b2
    result = np.dot(X, w2) + b2
    if result >= 0:
        return 1
    else:
        return 0

def or_predict(x1, x2):
    X = np.c_[x1, x2]
    result = np.dot(X, w3) - b3
    if result >= 0:
        return 1
    else:
        return 0

def xor_predict(x1, x2):
    or_layer = or_predict(x1, x2)
    nand_layer = nand_predict(x1, x2)
    # X = np.c_[or_layer, nand_layer]
    result = and_predict(nand_layer, or_layer)
    return result
    # result = np.dot(X, w1) - b1
    # if result >= 0:
    #     return 1
    # else:
    #     return 0

In [43]:
def step(x):
    return np.where(x > 0, 1, 0)

def xor_predict(x1, x2):
    X = np.c_[x1, x2]
    w_layer1 = np.c_[w3, w2]
    b_layer1 = np.concatenate([b3, b2])
    layer1 = step(np.dot(X, w_layer1) - b_layer1)
    layer2 = np.dot(layer1, w1) - b1
    result = step(layer2)
    return result

In [58]:
w_1 = np.c_[w3, w2]
b_1 = np.concatenate([b3, b2])

layers = [(w_1, b_1), (w1, b1)]

def predict_xor(x1, x2):
    X = np.c_[x1, x2]
    w, b = layers[0]
    result = np.dot(X, w) - b
    result = step(result)
    for w, b in layers:
        result = np.dot(result, w) - b
        result = step(result)
    return result
        
predict_xor(1, 0)


array([[0]])

In [99]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
import tensorflow
# import matplotlib.pyplot as plt
import PIL
import warnings
warnings.filterwarnings('ignore')

In [124]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)
input_shape = X_train.shape[1:]

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
# output_shape = Y_train.shape[1:][0]
output_shape = Y_train.shape[1]

model = Sequential(
    [
        # Flatten(input_shape=input_shape),
        Dense(1024, activation='relu', input_shape=input_shape),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(output_shape, activation='softmax')
    ]
)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=20, batch_size=512)

Epoch 1/20
118/118 [==============================] - 3s 16ms/step - loss: 3.6154 - accuracy: 0.7829
Epoch 2/20
118/118 [==============================] - 1s 12ms/step - loss: 0.1956 - accuracy: 0.9424
Epoch 3/20
118/118 [==============================] - 1s 9ms/step - loss: 0.1171 - accuracy: 0.9642
Epoch 4/20
118/118 [==============================] - 1s 10ms/step - loss: 0.0760 - accuracy: 0.9770
Epoch 5/20
118/118 [==============================] - 1s 11ms/step - loss: 0.0539 - accuracy: 0.9837
Epoch 6/20
118/118 [==============================] - 2s 13ms/step - loss: 0.0369 - accuracy: 0.9888
Epoch 7/20
118/118 [==============================] - 1s 10ms/step - loss: 0.0267 - accuracy: 0.9920
Epoch 8/20
118/118 [==============================] - 1s 9ms/step - loss: 0.0163 - accuracy: 0.9954
Epoch 9/20
118/118 [==============================] - 1s 10ms/step - loss: 0.0122 - accuracy: 0.9966
Epoch 10/20
118/118 [==============================] - 1s 9ms/step - loss: 0.0084 - accuracy:

In [143]:
num = 1

layers = model.layers
wb = []
ly = []
a = X_test[[num]]
for i in layers:
    wb.append((i.get_weights()))
    tmp = np.dot(a, wb[-1][0]) + wb[-1][1]
    ly.append(tmp)
    a = tmp
print(np.argmax(ly[-1]))

print(np.argmax(model.predict(X_test[[num]])))
# w1, b1 = layers[1].get_weights()
# w2, b2 = layers[2].get_weights()
# w3, b3 = layers[3].get_weights()
# x = X_test[[0]]
# layer1 = np.dot(x, w1) + b1
# layer2 = np.dot(layer1, w2) + b2
# layer3 = np.dot(layer2, w3) + b3
# np.argmax(layer3[0])

2
1


In [302]:
def relu(x):
    return np.where(x > 0, x, 0)
def sigmoid(x):
    # return 1 / (1+np.e ^ (-x))
    return 1 / (1+np.exp(-x))
def softmax(x):
    if x.ndim == 1:
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()
    else:
        # result = np.zeros_like(x)
        result = []
        for i in range(x.shape[0]):
            # result[i] = np.exp(x[i]) / np.sum(np.exp(x[i]))
            e_x = np.exp(x[i] - np.max(x[i]))
            result.append(e_x / np.sum(e_x))
        result = np.array(result)
        return result
    # return np.exp(x) / np.sum(np.exp(x))

def softmax2(x):
    x = (x.T - np.max(x, axis=1)).T
    x = (np.exp(x).T / np.sum(np.exp(x), axis=1)).T
    return x


activation = {'relu': relu, 'sigmoid': sigmoid, 'softmax': softmax}

def predict(x):
    layers = model.layers
    w1, b1 = layers[0].get_weights()
    result = np.dot(x, w1) + b1
    # result = relu(result)
    result = layers[0].activation(result)
    for i in layers[1:-1]:
        w, b = i.get_weights()
        result = np.dot(result, w) + b
        # result = relu(result)
        result = activation[i.activation.__name__](result)
        result = i.activation(result)
    result = np.dot(result, layers[-1].get_weights()[0]) + layers[-1].get_weights()[1]
    return np.argmax(result)

num = 0
print(predict(X_test[[num]]), np.argmax(model.predict(X_test[[num]])))


7 7


In [285]:
np.ravel(X_train[[0]]).reshape(1, -1).shape

(1, 784)

In [305]:
def predict(X):
    layers = model.layers
    # x = np.ravel(X).reshape(1, -1)
    x = np.ravel(X).reshape(X.shape[0], -1)
    w, b = layers[0].get_weights()
    pred = np.dot(x, w) + b
    pred = layers[0].activation(pred)
    for i in layers[1:-1]:
        w, b = i.get_weights()
        pred = np.dot(pred, w) + b
        pred = i.activation(pred)
    w, b = layers[-1].get_weights()
    pred = np.dot(pred, w) + b
    pred = softmax2(pred)
    # return layers[-1].activation, pred
    # return np.argmax(pred)
    return np.argmax(pred, axis=1)
    
def predict_2(X):
    layers = model.layers
    if X.shape[0] == 1:
        x = np.ravel(X).reshape(1, -1)
        w, b = layers[0].get_weights()
        pred = np.dot(x, w) + b
        pred = layers[0].activation(pred)
        for i in layers[1:-1]:
            w, b = i.get_weights()
            pred = np.dot(pred, w) + b
            pred = i.activation(pred)
        w, b = layers[-1].get_weights()
        pred = np.dot(pred, w) + b
        pred = softmax(pred)
        # return layers[-1].activation, pred
        # return np.argmax(pred)
        return np.argmax(pred, axis=1)
    else:
        result = []
        for i in range(X.shape[0]):
            x = np.ravel(X[i]).reshape(1, -1)
            result.append(predict(x))
        return result

In [306]:
predict(X_test[0:5])

array([7, 2, 1, 0, 4], dtype=int64)

In [326]:
x = np.array([[1, 2, 3], [3, 2, 7]])
softmax2(x)

array([[0.09003057, 0.24472847, 0.66524096],
       [0.01786798, 0.00657326, 0.97555875]])

In [26]:
class Model:
    def __init__(self, input_shape, x):
        self.model = []
        self.input_shape = input_shape
        self.x = x
        w = np.random.randn(self.input_shape, x)
        b = np.random.randn(x)
        self.model.append((w, b))

    def add(self, x):
        self.input = self.model[-1][1].size
        w = np.random.randn(self.input, x)
        b = np.random.randn(x)
        self.model.append((w, b))
    
    def summary(self):
        print('Model: "sequential_1"')
        print('_________________________________________________________________')
        print('Layer (type)                 Output Shape              Param #   ')
        print('=================================================================')
        for i, j in enumerate(self.model):
            print(f'layer {i}   {j[0].shape}')

a = Model(784, 100)
a.add(50)
a.add(50)
a.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer 0   (784, 100)
layer 1   (100, 50)
layer 2   (50, 50)


In [6]:
from Activation import Activation
import numpy as np
x = np.array([[1, 2, 3], [3, 2, -7]])
b = Activation()
b.relu(x)

array([[1, 2, 3],
       [3, 2, 0]])

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(30, )))
model.add(Dense(200, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile()

model.layers[0].get_weights()[0].shape
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               3100      
_________________________________________________________________
dense_4 (Dense)              (None, 200)               20200     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                2010      
Total params: 25,310
Trainable params: 25,310
Non-trainable params: 0
_________________________________________________________________
